In [62]:
import pandas as pd 
import pyarrow as pa
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm  import tqdm
import numpy as np



In [63]:
sent_and_word=pd.read_excel('labeled_dataset.xlsx')

/Users/anisha/Documents/Booz/bias_project/bias/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [64]:
sent_and_word.head()

,Unnamed: 0,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words
0,0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1,1,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']"
1,1,The increasingly bitter dispute between Americ...,https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1,1,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",['bitter']
2,2,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1,1,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis']
3,3,A professor who teaches climate change classes...,https://www.breitbart.com/politics/2019/05/09/...,breitbart,environment,right,1,1,Non-biased,No agreement,A professor who teaches climate change classes...,['legitimate']
4,4,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1,1,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '..."


In [65]:
sent=sent_and_word[['sentence', 'Label_bias']]

In [66]:
word=sent_and_word[['sentence', 'biased_words']]

In [67]:
sent.head()

,sentence,Label_bias
0,YouTube is making clear there will be no “birt...,Biased
1,The increasingly bitter dispute between Americ...,Non-biased
2,So while there may be a humanitarian crisis dr...,Biased
3,A professor who teaches climate change classes...,Non-biased
4,"Looking around the United States, there is nev...",Biased


In [68]:
word.head()

,sentence,biased_words
0,YouTube is making clear there will be no “birt...,"['belated', 'birtherism']"
1,The increasingly bitter dispute between Americ...,['bitter']
2,So while there may be a humanitarian crisis dr...,['crisis']
3,A professor who teaches climate change classes...,['legitimate']
4,"Looking around the United States, there is nev...","['killing', 'never', 'developing', 'humans', '..."


In [69]:
sent=sent[sent['Label_bias'] != 'No agreement']

In [70]:
sent['Label_bias'].value_counts()

Label_bias
Biased        1018
Non-biased     533
Name: count, dtype: int64

In [72]:
sent.to_csv('labeled_dataset_sent.csv', index=False)

In [73]:
word.to_csv('labeled_dataset_word.csv', index=False)

In [74]:
minority_class_size = sent[sent['Label_bias'] == 'Non-biased'].shape[0]

# Sample from majority class
majority_downsampled = sent[sent['Label_bias'] == 'Biased'].sample(
    n=minority_class_size, 
    random_state=42
)

# Combine with minority class
sent= pd.concat([
    sent[sent['Label_bias'] == 'Non-biased'],
    majority_downsampled
])

In [75]:
sent.to_csv('balanced_dataset_sent.csv', index=False)

In [77]:
word.to_csv('labeled_dataset_word.csv', index=False)

In [76]:
sent.rename(columns={"sentence": "text", "Label_bias": "label"}, inplace=True)

In [ ]:
#word.rename(columns={"sentence": "text", "biased_words": "label"}, inplace=True)

In [78]:
num={'Non-biased': 0, 'Biased': 1}

In [79]:
sent['label']=sent['label'].replace(num)


/var/folders/xq/q8k6p6r524x9md10q7frq9dr0000gn/T/ipykernel_22967/3207431553.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sent['label']=sent['label'].replace(num)


In [80]:
word.head()

,sentence,biased_words
0,YouTube is making clear there will be no “birt...,"['belated', 'birtherism']"
1,The increasingly bitter dispute between Americ...,['bitter']
2,So while there may be a humanitarian crisis dr...,['crisis']
3,A professor who teaches climate change classes...,['legitimate']
4,"Looking around the United States, there is nev...","['killing', 'never', 'developing', 'humans', '..."


In [81]:
hg_dataset_sent = Dataset(pa.Table.from_pandas(sent))


In [82]:
#hg_dataset_word = Dataset(pa.Table.from_pandas(word))

In [83]:
shuffled_dataset_sent = hg_dataset_sent.shuffle(seed=42)

In [84]:
len(shuffled_dataset_sent)

1066

In [85]:
train_test_split_sent = shuffled_dataset_sent.train_test_split(test_size=0.2, seed=42)

In [86]:
train_test_split_sent

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 852
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 214
    })
})

In [87]:
train_dataset_sent = train_test_split_sent['train']
test_dataset_sent = train_test_split_sent['test']

In [88]:
test_set_pandas=test_dataset_sent.to_pandas()

In [89]:
test_set_pandas.to_csv('test_set_sent.csv', index=False)

In [90]:
test_set_pandas['label'].dtype

dtype('int64')

In [91]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [92]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset_sent.map(preprocess_function, batched=True)
tokenized_test = test_dataset_sent.map(preprocess_function, batched=True)

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

In [47]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [48]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [50]:
training_args = TrainingArguments(
    output_dir='results',
    learning_rate=2e-5,
    num_train_epochs=4,                    # +1 from your original
    per_device_train_batch_size=16,        # Will auto-adjust based on GPU
    per_device_eval_batch_size=32,         # Larger for eval (no gradients)
    warmup_ratio=0.1,
    weight_decay=0.01,                     # Reduced from your 0.05
    eval_strategy="epoch",           # Changed from "epoch"
    eval_steps=200,
    save_strategy="epoch",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1",            # 🔥 Key change for bias detection
    greater_is_better=True,                            # 2x speedup
    seed=42,
    push_to_hub=False
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

/var/folders/xq/q8k6p6r524x9md10q7frq9dr0000gn/T/ipykernel_22967/1698321876.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.530277,0.785047,0.785179,0.785448,0.785047
2,No log,0.467261,0.761682,0.760112,0.762646,0.761682
3,No log,0.482240,0.761682,0.760112,0.762646,0.761682
4,No log,0.499418,0.771028,0.769108,0.773026,0.771028


TrainOutput(global_step=216, training_loss=0.4376838472154405, metrics={'train_runtime': 232.7536, 'train_samples_per_second': 14.642, 'train_steps_per_second': 0.928, 'total_flos': 64067650379664.0, 'train_loss': 0.4376838472154405, 'epoch': 4.0})